In [1]:
# required libraries
import os
import pandas as pd
import numpy as np
from pdfminer import high_level

train = pd.read_csv('dataset/train.csv')
test = pd.read_csv('dataset/test.csv')

#paths
train_path = "dataset/trainResumes/"
test_path = "dataset/testResumes/"

# epty list for resumes text
train_resumes = []
test_resumes = []

# ids
ids = list(train.CandidateID)
test_ids = list(test.CandidateID)

# pdf2string
def pdf2string_train(path, ids, resumes):
    for i in ids:
        main_path = path+i+'.pdf'
        text = high_level.extract_text(main_path)
        str_list = text.split()
        str_list = str_list[:]
        string = ' '.join(str_list)
        resumes.append(string)
        

def pdf2string_test(path, test_ids, resumes):
    for i in test_ids:
        main_path = path+i+'.pdf'
        text = high_level.extract_text(main_path)
        str_list = text.split()
        str_list = str_list[:]
        string = ' '.join(str_list)
        resumes.append(string)


pdf2string_train(train_path, ids, train_resumes)
pdf2string_test(test_path,  test_ids, test_resumes)


In [16]:
train_resumes[0]

'L I A M A N D R E W S F R E S H E R WORK EXPERIENCE DICTIS making. Intern Trainee, Jan 2020 to Apr 2020 Responsible for performing and helping in decision EXECUTIVE SUMMARY Fresher with strong statistical and analytic capabilities. Someone who is driven by the passion for problem solving. Though I am from Civil Engineering background I have always been fascinated with data and how Machine Learning is evolving with iit. PERSONAL SKILLS Data Analyst, Data Mining, Data Visualization, Machine Learning, Linear Regression, Statistical Modeling, Predictive Modeling, SQL Server, Oracle, Python. PROJECTS EXTRA-CURRICULARS Data Preprocessing with Python Data Visualization with Power BI ACADEMIC PROFILE Railway Signals Determiner using relay weight system. 2020 B.Tech(Civil) Garodia Institute of Technosciences,'

In [19]:
single_words = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
def remove_single_word(text):
    resume = []
    for i in text.split():
        if i not in single_words:
            resume.append(i)
    return ' '.join(resume)

train_resumes_str = []
for resume in train_resumes:
    string = remove_single_word(resume)
    train_resumes_str.append(string)

test_resumes_str = []
for resume in test_resumes:
    string = remove_single_word(resume)
    test_resumes_str.append(string)

In [21]:
train_resumes_str[1]

"Ellie Mackey Executive Profile Work Experience BB House of Developers Intern Trainee, Nov 2019 - Feb 2020 Learned the in's and out's of software development of machine learning projects. Education B.CA St. Subhash College, Hyderabad 2020. Actively seeking job. An enthusiast, with a Master's degree in Electronics & Telecommunication, very keen to learn & eagerly looking for opportunities. Skills Data science, Deep learning, Neural Networks, Artificial Intelligence, Natural Language Processing, Pattern Recognition, Machine Learning, Predictive Analytics, Logistic regression. Other Activities MACHINE LEARNING with Python. Projects Mine and analyze data from databases to drive optimization and improvement of product development, marketing techniques and business strategies."

In [11]:
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

train_resumes_lower = []
for resume in train_resumes_str:
    train_resumes_lower.append(resume.lower())

test_resumes_lower = []
for resume in test_resumes_str:
    test_resumes_lower.append(resume.lower())

PUNCT_TO_REMOVE = string.punctuation
#print(PUNCT_TO_REMOVE)
#print(type(PUNCT_TO_REMOVE))
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

train_punc_removed = []
for resume in train_resumes_lower:
    punc_removed = remove_punctuation(resume)
    train_punc_removed.append(punc_removed)

test_punc_removed = []
for resume in test_resumes_lower:
    punc_removed = remove_punctuation(resume)
    test_punc_removed.append(punc_removed)

STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

train_stopwords_removed = []
for resume in train_punc_removed:
    stopwords_removed = remove_stopwords(resume)
    train_stopwords_removed.append(stopwords_removed)

test_stopwords_removed = []
for resume in test_punc_removed:
    stopwords_removed = remove_stopwords(resume)
    test_stopwords_removed.append(stopwords_removed)

lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

train_lemma = []
for resume in train_stopwords_removed:
    lemma = lemmatize_words(resume)
    train_lemma.append(lemma)

test_lemma = []
for resume in test_stopwords_removed:
    lemma = lemmatize_words(resume)
    test_lemma.append(lemma)



In [12]:
train_lemma[0]

'l n r e w f r e h e r work experience dictis make intern trainee jan 2020 apr 2020 responsible perform help decision executive summary fresher strong statistical analytic capability someone drive passion problem solve though civil engineering background always fascinate data machine learn evolve iit personal skill data analyst data mining data visualization machine learn linear regression statistical model predictive modeling sql server oracle python project extracurriculars data preprocessing python data visualization power bi academic profile railway signal determiner use relay weight system 2020 btechcivil garodia institute technosciences'

In [13]:
train_str[0]

'                                    2 0 2 0     2 0 2 0                                                                                                                                     2 0 2 0        '

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

train = pd.read_csv('dataset/train.csv')
test = pd.read_csv('dataset/test.csv')

train_df = pd.concat([train, pd.DataFrame(train_lemma, columns=['resumes'])], axis = 1)
test_df = pd.concat([test, pd.DataFrame(test_lemma, columns=['resumes'])], axis = 1)

print(train_df.head())
print(test_df.head())

tfidf = TfidfVectorizer(max_features=10000, 
                        strip_accents='unicode', 
                        analyzer='word',
                        lowercase=False,
                        ngram_range=(1, 1), 
                        stop_words = 'english')

tfidf_matrix_train = tfidf.fit_transform(train_df['resumes'])
tfidf_matrix_test = tfidf.transform(test_df['resumes'])
print(tfidf_matrix_train.shape)
print(tfidf_matrix_test.shape)

     CandidateID  Match Percentage  \
0  candidate_011             13.60   
1  candidate_113             36.63   
2  candidate_123             54.93   
3  candidate_012             41.46   
4  candidate_002             48.91   

                                             resumes  
0  l n r e w f r e h e r work experience dictis m...  
1  ellie mackey f r e h e r n e r n executive pro...  
2  f e l x w n n fresher skills project activites...  
3  jimmy gartner n g e r professional profile emp...  
4  n q u r associate analyst skill certify data a...  
     CandidateID                                            resumes
0  candidate_014  grace bailry c h n e l e r n n g e v e l p e n...
1  candidate_098  l e l e u n software engineer skill assistant ...
2  candidate_075  keiron pavard e n g n e e r personal profile w...
3  candidate_016  e l r n c e n c e j r work experience na acade...
4  candidate_131  zachary perez n e r n p r f l e k l l good kno...
(90, 1873)
(60, 1873)


In [4]:
import xgboost as XGB

In [5]:
y = train_df['Match Percentage']

In [6]:
from sklearn.model_selection import GridSearchCV
xgb_grid = {'learning_rate': [0.1, 0.3, 0.2, 0.01, 0.02, 0.03, 0.04, 0.05, 0.07, 0.005, 0.006, 0.007, 0.0075, 0.008],
            'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200],
            'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 20]}


xgb = GridSearchCV(XGB.XGBRegressor(),
                    param_grid = xgb_grid,
                    cv=2,
                    verbose=True)
xgb.fit(tfidf_matrix_train, y)
xgb.best_params_

Fitting 2 folds for each of 2184 candidates, totalling 4368 fits


{'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 1000}

In [67]:

xgb = XGB.XGBRegressor(learning_rate=0.005, 
                        n_estimators=700, 
                        objective='reg:squarederror', 
                        max_depth=8, 
                        reg_lambda = 1.3,
                        gamma = 1,
                        min_child_weight =1.5,
                        max_delta_step = 100,
                        random_state = 31).fit(tfidf_matrix_train, y)

In [153]:
from lightgbm import LGBMRegressor
lgbm = LGBMRegressor(num_leaves=31,
                    learning_rate = 0.01,
                    n_estimators = 1000,
                    reg_lambda = 2.5,
                    reg_alpha = 2,
                    random_state=31).fit(tfidf_matrix_train, y)

In [168]:
from catboost import CatBoostRegressor
cat = CatBoostRegressor().fit(tfidf_matrix_train, y)

Learning rate set to 0.025425
0:	learn: 15.8268917	total: 187ms	remaining: 3m 6s
1:	learn: 15.7305636	total: 202ms	remaining: 1m 40s
2:	learn: 15.6811411	total: 212ms	remaining: 1m 10s
3:	learn: 15.5344921	total: 222ms	remaining: 55.3s
4:	learn: 15.4083514	total: 232ms	remaining: 46.3s
5:	learn: 15.3023917	total: 243ms	remaining: 40.3s
6:	learn: 15.2062998	total: 251ms	remaining: 35.6s
7:	learn: 15.1151314	total: 264ms	remaining: 32.7s
8:	learn: 14.9932991	total: 273ms	remaining: 30.1s
9:	learn: 14.8506554	total: 281ms	remaining: 27.9s
10:	learn: 14.7456651	total: 290ms	remaining: 26.1s
11:	learn: 14.6122384	total: 299ms	remaining: 24.6s
12:	learn: 14.5360673	total: 307ms	remaining: 23.3s
13:	learn: 14.4353127	total: 316ms	remaining: 22.2s
14:	learn: 14.3088053	total: 324ms	remaining: 21.3s
15:	learn: 14.1968794	total: 332ms	remaining: 20.4s
16:	learn: 14.0876313	total: 341ms	remaining: 19.7s
17:	learn: 13.9662178	total: 349ms	remaining: 19.1s
18:	learn: 13.8431969	total: 358ms	remaini

In [169]:
def submission(model, test_sentences):
    test1 = pd.read_csv('dataset/test.csv')
    preds = model.predict(test_sentences)
    prediction = pd.DataFrame(preds, columns = ['Match Percentage'])
    sub_df = pd.concat([test1, prediction], axis = 1)
    return sub_df

sub = submission(cat, tfidf_matrix_test)
sub.to_csv('submission file/cat sub.csv')
print(sub.head(10))

     CandidateID  Match Percentage
0  candidate_014         33.586811
1  candidate_098         37.720325
2  candidate_075         38.840289
3  candidate_016         38.542037
4  candidate_131         43.224407
5  candidate_056         42.104137
6  candidate_141         48.851910
7  candidate_044         46.867703
8  candidate_029         37.251768
9  candidate_120         43.437835


In [148]:
print(sub.head(10))

     CandidateID  Match Percentage
0  candidate_014         23.700393
1  candidate_098         32.999942
2  candidate_075         38.796772
3  candidate_016         35.225443
4  candidate_131         33.061261
5  candidate_056         35.019762
6  candidate_141         48.173307
7  candidate_044         55.757834
8  candidate_029         31.043669
9  candidate_120         36.911600
